### Tool Calling

Tool calling is the capability to give the model access to external functionalities and APIs. In this notebook we will be walking through the concept and best practices for using this capability with Nova models.

### Defining the Schema

When we refer to a "tool" we are referring to a function that will execute actual code. To provide the details about this function, you will provide a tool configuration to the model. This tool configuration will contain details such as the name, description and details about the parameters. 

You can imagine a calculator tool that might be defined as:
```python
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "calculator", # Name of the tool
                "description": "A calculator tool that can execute a math equation",
                "inputSchema": {
                    "json": { 
                        "type": "object", # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "equation": { # The name of the parameter
                                "type": "string", # parameter type: string/int/etc
                                "description": "The full equation to evaluate" # Helpful description of the parameter
                            }
                        },
                        "required": [ # List of all required parameters
                            "equation"
                        ]
                    }
                }
            }
        }
    ]
}
```

or a retriever tool that allows you to search SEC filings

```python
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "secRetriever", 
                "description": "A retriever that can access SEC filings from a database",
                "inputSchema": {
                    "json": { 
                        "type": "object", # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "query": { 
                                "type": "string", 
                                "description": "The full query to search for" 
                            },
                            "ticker": { 
                                "type": "string", 
                                "description": "The stock ticker of the company"
                            },
                            "year": { 
                                "type": "string", 
                                "description": "The relevant year of the filings"
                            }
                        },
                        "required": [ # Note that year is not provided, this indicates it's an optional parameter
                            "query",
                            "ticker"
                        ]
                    }
                }
            }
        }
    ]
}
```

or a multiplication tool that takes two integers and multiplies them

```python
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "multiply",
                "description": "Multiplies two numbers together",
                "inputSchema": {
                    "json": { 
                        "type": "object", # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "int1": { 
                                "type": "integer", 
                                "description": "The first number to multiply" 
                            },
                            "int2": { 
                                "type": "integer", 
                                "description": "The second number to multiply"
                            }
                        },
                        "required": [
                            "int1",
                            "int2"
                        ]
                    }
                }
            }
        }
    ]
}

```


## Prerequisites

Run the cells in this section to install the packages needed by the notebooks in this workshop. ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

_IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts._

In [ ]:
# Install Strands Agents
%pip install -q --force-reinstall "botocore>=1.40.26" "boto3>=1.40.26" "awscli>=1.29.57"

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Provide the Tool to Nova
In these examples we will be using the converse API and you can pass the tools to the model through the toolConfig parameter on the model. When we are utilizing tool calling - we recommend taking advantage of "greedy decoding" parameters. With Nova this is done by setting the temperature, topP and topK to 1.

Starting with the calculator:


In [ ]:
import boto3
from botocore.config import Config
import json
import os
from pprint import pprint


LITE_MODEL_ID = "us.amazon.nova-2-lite-v1:0"

# Create a Bedrock Runtime client
client = boto3.client("bedrock-runtime", 
                      region_name="us-east-1", 
                      config=Config(read_timeout=10000))


In [ ]:
request = {
  "modelId": LITE_MODEL_ID,  
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "text": "evaluate 7**6 by using the calculator tool "
        }
      ]
    }
  ],
  "toolConfig": {
    "tools": [
        {
            "toolSpec": {
                "name": "evaluate_equation",
                "description": "A calculator tool that can execute a math equation",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "equation": {
                                "type": "string",
                                "description": "The full equation to evaluate",
                            }
                        },
                        "required": ["equation"],
                    }
                },
            }
        }
    ]
  },
  "inferenceConfig": {
    "temperature": 1,
    "topP": 0.9,
    "maxTokens": 10000
  },
  "additionalModelRequestFields": {
    "reasoningConfig": {
      "type": "enabled",
      "maxReasoningEffort": "low"
    }
  }
}
try:
  response = client.converse(**request)
  pprint(response)
except Exception as e:
  print(f'Request ID: {e.response["ResponseMetadata"]["RequestId"]}')
  print(e)

In [ ]:
stop_reason = response["stopReason"]
tool = next(
        block["toolUse"]
        for block in response["output"]["message"]["content"]
        if "toolUse" in block
    )
tool['input']['equation'], tool['name']

In [ ]:
def evaluate_equation(equation_string):
    """
    Evaluates a mathematical equation provided as a string.

    Args:
        equation_string (str): The mathematical equation to evaluate.

    Returns:
        The result of the evaluated equation, or an error if the
        equation is invalid or contains unsafe operations.
    """
    try:
        result = eval(equation_string)
        print(result)
        return result
    except (SyntaxError, NameError, TypeError) as e:
        return f"Error evaluating equation: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"
if tool['name'] == 'evaluate_equation':
    evaluate_equation(tool['input']['equation'])


#### Tool Choice - Auto 
#### Tool - Nova Grounding

For use cases where a tool isn't always required - you can set the tool choice to auto. This is the default behavior and will leave the tool selection completely up to the model.

In [ ]:
## In this example, we will specify two system tools and let Nova decide which tool to pick based on the user query 
tool_config = {
    "toolChoice": {"auto": {}},
    "tools": [
        {
            "systemTool": {
                "name": "nova_grounding"
            }
        },
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ],
}

messages = [
    {
        "role": "user",
        "content": [
            {"text": "Who was in the cast of movie Wicked"},
        ],
    }
]

system = [{"text": "You are a helpful chatbot. You can use a tool if necessary"}]
inf_params = {"maxTokens": 30000} 
try:
    response = client.converse(
        modelId=LITE_MODEL_ID,
        system=system,
        messages=messages,
        inferenceConfig=inf_params,
        additionalModelRequestFields={
            "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "low"
            },
        },
        toolConfig=tool_config,
    )

    for text in response['output']['message']['content']:
        if 'text' in text.keys():
            text = text['text']
            print(text)
    #pprint(response['output']['message']['content'], indent=4)
except Exception as e:
  print(f'Request ID: {e.response["ResponseMetadata"]["RequestId"]}')
  print(e)

In [ ]:
## Run the following cell to see entire resposne payload. Note the output may be truncated 
## you may need to adjust settings, or view it as scrollable element in order to view entire output
pprint(
response['output']['message']['content']
)

### Nova's Built-in tool: Code Interpreter

In [ ]:

request = {
  "modelId": LITE_MODEL_ID,  
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "text": "Calculate 7^6 using the code interpreter"
        }
      ]
    }
  ],
  "toolConfig": {
    "tools": [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]
  },
  "inferenceConfig": {
    "temperature": 1,
    "topP": 0.9,
    "maxTokens": 10000
  },
  "additionalModelRequestFields": {
    "reasoningConfig": {
      "type": "enabled",
      "maxReasoningEffort": "low"
    }
  }
}

try:
  response = client.converse(**request)
  pprint(response['output']['message']['content'][4], indent=2)
except Exception as e:
  print(f'Request ID: {e.response["ResponseMetadata"]["RequestId"]}')
  print(e)


### Nova's Built-in tool: Web Search 

In [ ]:
request = {
  "modelId": LITE_MODEL_ID,
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "text": "What teams played in 2025 MLB world series?"
        }
      ]
    }
  ],
  "toolConfig": {
        "tools": [
            {
                "systemTool": {
                    "name": "nova_grounding"
                }
            }
        ]
  },
  "inferenceConfig": {
    "temperature": .7,
    "topP": 0.9,
    "maxTokens": 10000 # Make sure to set a large max tokens to accommodate reasoning..
  },
  "additionalModelRequestFields": {
    "reasoningConfig": {
      "type": "enabled",
      "maxReasoningEffort": "low"
    }
  }
}
try:
  response = client.converse(**request)
  pprint(response['output']['message']['content'][4], indent=2)
except Exception as e:
  print(f'Request ID: {e.response["ResponseMetadata"]["RequestId"]}')
  print(e)
